In [ ]:
"""Rotational Barriers"""

__authors__ = ["Olaseni Sode"]
__email__   = ["osode@calstatela.edu"]
__date__    = "2024-09-10"

## Rotational Barrier Potential Energy Scan

In this exercise, you will create one-dimensional potential energy surfaces (PES) of the rotational barrier for ethane and butane. This will help you visualize the Born-Oppenheimer potential energy surface of a molecule and identify global and local minima and maxima. You will use Python and Psi4 to perform electronic structure calculations and generate PES plots.

In [ ]:
# import the python modules that we will use
import psi4
import numpy as np

## 1. Rigid Potential Energy Scan


#### Step 1: Define Ethane Molecule in Z-Matrix Format

To begin, we need to construct the ethane molecule in Z-matrix format. A Z-matrix is a way of specifying molecular geometry that uses internal coordinates (bond lengths, bond angles, and dihedral angles) rather than Cartesian coordinates (x, y, z).

In [ ]:
# set the molecule name for your files and plots
molecule_name = "ethane"

# enter starting coordinates in z-matrix format
geometry = """
C
C 1 1.5307
H 1 1.0936 2 111.3578
H 1 1.0936 2 111.3578 3 120
H 1 1.0936 2 111.3578 3 240
H 2 1.0936 1 111.3578 3 60
H 2 1.0936 1 111.3578 3 180
H 2 1.0936 1 111.3578 3 300
"""
molecule = psi4.geometry(geometry)

#### Step 2: Perform Energy Calculation

Now that the ethane molecule is set up, we can run a quick Hartree-Fock calculation to obtain its energy.


In [ ]:
# set the amount of memory that you will need
psi4.set_memory('2 GB')

# set the number of threads (processes) for Psi4 to use
# (~all modern computers can handle 2 threads)
psi4.set_num_threads(2)

# calculate the initial energy of the molecule using the Hartree-Fock method 
# and the cc-pVDZ basis set; print this output to a file
psi4.set_output_file(molecule_name + '_energy_initial.dat', False)
E = psi4.energy('scf/cc-pVDZ')

#print the energy out to the notebook
print('The energy is: ', E)

#### Step 3: Define the Dihedral Scan Parameters

Now you will rotate the central C-C torsion angle (a.k.a. $\phi$) steadily increasing from 0$^\circ$ to 360$^\circ$ in increments of 5$^\circ$ (i.e., a total of 72 calculations).  To do this we will use a `for` loop to iterate over the range of values of $\phi$, calculating and recording the energy value at each iteration.

These calculations will take a bit of time, so be patient! To help speed up the process, we are going to switch to using the 3-21G basis set, which is considerably smaller than the cc-pVDZ basis set that we have used up to this point. You will know all of the computations are complete when you see "All calculations complete!".

**Potentially useful documentation links**
- The Python function `range()`: https://docs.python.org/3/library/stdtypes.html?highlight=range#range)
- Python list examples: https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-stacks

In [ ]:
x = []
y = []

dihedrals = np.arange (0,360, 5)
for d in dihedrals:
    geometry = f"""
    C
    C 1 1.5307
    H 1 1.0936 2 111.3578
    H 1 1.0936 2 111.3578 3 120
    H 1 1.0936 2 111.3578 3 240
    H 2 1.0936 1 111.3578 3 {d}
    H 2 1.0936 1 111.3578 3 {d+120}
    H 2 1.0936 1 111.3578 3 {d+240}
    """

    ethane = psi4.geometry(geometry)
    E = psi4.energy('scf/cc-pVDZ',molecule=ethane)

    x.append(d)
    y.append(E)
    
print('All calculations complete!')

#### Step 4: Plotting the Potential Energy Surface

Since you have all of the necessary information, you can plot how the energy changes at each value of $\phi$. To make the data more human-understandable, before creating the plot, write code that does the following:

1. Create a new variable `y_kcal` that gives the *relative* energies in kcal/mol by:
 * subtracting off the minimum energy value from the list of energies
 * multiplying the resulting relative energies by the conversion factor of 627.5 (see: https://cccbdb.nist.gov/hartree.asp)
**Hint:** To make life easier, it is recommended to convert `y` into a NumPy array: `np.array(y)`


2. Create a scatter plot of relative energy (*y* axis) vs. phi (*x* axis). Make this plot look professional by customizing your axes limits, ticks, and labels using functions like `plt.xlim()`, `plt.xticks()`, and `plt.xlabel()` (and the analogous functions for the *y* axis).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# write code to convert absolute energies to relative energies
# by subtracting the minimum energy from the set; then convert
# from Hartrees to kcal/mol

y_kcal = y

# create a scatter plot of relative energy vs. phi
plt.scatter(x, y_kcal, color='MediumVioletRed')
plt.xlim(0, 360)

# beautify the plot by adding labels
plt.xlabel("### INSERT X LABEL ###")
plt.ylabel("### INSERT Y LABEL ###")
plt.title("### INSERT TITLE ###")
plt.show()

#### Step 5: Visualize the molecule

In [ ]:
import nglview as nv

structure = nv.Psi4Structure(ethane)
image = nv.NGLWidget(structure)
image

#### Question: **<span style="color:red">Can you identify the stationary states for the ethane molecule?</span>**
#### Answer:

## 2. Relaxed Potential Energy Scan

A relaxed potential energy scan (PES) allows us to study how the energy of a molecule changes as a specific dihedral angle is varied, while allowing all other geometric parameters (like bond lengths and bond angles) to adjust to their lowest energy configuration at each step. This method provides a more realistic picture of the rotational barrier compared to a rigid scan, where only one dihedral is modified while everything else is held constant.

#### Step 1: Setting up the Butane Molecule

**Build the Butane Molecule:** 
   - Open WebMO or IQMol, which are molecular modeling tools that allow you to construct molecules interactively.
   - Select the option to create a new molecule and start by placing the four carbon atoms in a straight chain to represent the butane backbone. 
   - Add the appropriate number of hydrogen atoms to each carbon to satisfy their valency (each carbon should have four bonds in total).

**Example Z-Matrix for Butane:**
   
Here is an example Z-matrix for a generic molecule. Make sure to replace this with your Z-matrix for butane.

`
C
C 1 B1
C 2 B2 1 A1
C 3 B3 2 A2 1 D1
H 1 B4 2 A3 3 D2
H 1 B5 2 A4 3 D3
H 1 B6 2 A5 3 D4
...
`

**Key Components:**
  - Bond Lengths (B1, B2, B3, etc.): The distances between atoms, measured in Angstroms.
  - Bond Angles (A1, A2, A3, etc.): The angles between three connected atoms, measured in degrees.
  - Dihedral Angles (D1, D2, D3, etc.): The angles between two planes formed by four atoms, which we will scan in the next steps.

**Verify the Z-Matrix:**
   - Double-check the Z-matrix to ensure that all atoms are correctly defined and that there are no mistakes in the bond lengths, bond angles, or dihedral angles.
   - If there are errors, correct them before proceeding to the next step.

By following these steps, you will set up the butane molecule correctly for the relaxed potential energy scan. The accuracy of this setup is crucial because any errors in the molecular geometry will directly affect the calculated potential energy surface.

In [ ]:
# set the molecule name for your files and plots
molecule_name = "butane"

# enter starting coordinates in z-matrix format
geometry = """
### ADD ZMATRIX HERE ###
"""
butane = psi4.geometry(geometry)
E = psi4.optimize('scf/3-21g', molecule=butane)

#### Step 2. Visualize the molecule

In [ ]:
import nglview as nv

structure = nv.Psi4Structure(butane)
image = nv.NGLWidget(structure)
image

#### Step 3. Setup and run relaxed scan
Now you will run a series of geometry optimizations to with the angle of the central C-C torsion (a.k.a. $\psi$) steadily increasing from 0$^\circ$ to 180$^\circ$ in increments of 5$^\circ$ (i.e., a total of 36 calculations). To do this we will use a `for` loop to iterate over the range of values of $\psi$, calculating and recording the energy value at each iteration.

These calculations will take a bit of time, so be patient! To help speed up the process, we are going to switch to using the 3-21G basis set, which is considerably smaller than the cc-pVDZ basis set that we have used up to this point. You will know all of the computations are complete when you see "All optimizations complete!".

In [ ]:
# set the atoms involved in the C-C torsion
torsion = "1 2 3 4"

In [ ]:

x = []
y = []

start_angle = butane.get_variable("D1")

dihedrals = np.arange (0,181, 5)
for d in dihedrals:
    psi4.core.clean()
    
    butane.set_variable("D1",start_angle + d)
    torsion_constraint = torsion +" "+str(d + start_angle) 
    
    # sets the torsion constraint
    psi4.set_options({'optking__fixed_dihedral': torsion_constraint})
    
    # gau_loose convergence shortens the optimization time
    psi4.set_options({'optking__g_convergence': 'gau_loose'}) 
    
    E = psi4.optimize('scf/3-21g', molecule=butane)

    x.append(d + start_angle)
    y.append(E)
    
    print(d,E)
    
print('All optimizations complete!')

In [ ]:
structure = nv.Psi4Structure(butane)
image = nv.NGLWidget(structure)
image

#### Step. 4 Plotting how energy changes during the potential energy scan

Again, you can plot how the energy changes at each value of $\psi$. Before creating the plot, write code that gives the *relative* energies in kcal/mol.

Create a scatter plot of relative energy (*y* axis) vs. phi (*x* axis). Make this plot look professional by customizing your axes limits, ticks, and labels using functions like `plt.xlim()`, `plt.xticks()`, and `plt.xlabel()` (and the analogous functions for the *y* axis).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


# write code to convert absolute energies to relative energies
# by subtracting the minimum energy from the set; then convert
# from Hartrees to kcal/mol

y_kcal = _______________

# create a scatter plot of relative energy vs. phi
plt.scatter(x, y_kcal, color='MediumBlue')

# beautify the plot by adding labels
plt.xlabel("### INSERT X LABEL ###")
plt.ylabel("### INSERT Y LABEL ###")
plt.title("### INSERT TITLE ###")
plt.show()

#### Question: **<span style="color:red">Why did you only scan the dihedral angle from 0$^\circ$ to 180$^\circ$ instead of 0$^\circ$ to 360$^\circ$? What would the 0$^\circ$ to 360$^\circ$ plot look like compared to the one you plotted?</span>**

#### Answer:

#### Question: **<span style="color:red">Why was a relaxed scan used for the butane molecule and a rigid scan used for the ethane molecule?</span>**
#### Answer: